# Home Credit Default Risk

Can you predict how capable each applicant is of repaying a loan?

Many people struggle to get loans due to **insufficient or non-existent credit histories**. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

Home Credit strives to broaden financial inclusion for the **unbanked population by providing a positive and safe borrowing experience**. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

**Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.**

# Dataset

In [30]:
# #Python Libraries
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels
import pandas_profiling

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import time
import requests
import datetime

import missingno as msno
import math
import sys
import gc
import os

# #sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# #sklearn - metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score

# #XGBoost & LightGBM
import xgboost as xgb
import lightgbm as lgb

# #Missing value imputation
from fancyimpute import KNN, MICE

# #Hyperparameter Optimization
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

## Data Dictionary

In [2]:
!ls -l ../data/

total 2621364
-rw-r--r-- 1 karti 197609  26567651 May 17 18:06 application_test.csv
-rw-r--r-- 1 karti 197609 166133370 May 17 18:06 application_train.csv
-rw-r--r-- 1 karti 197609 170016717 May 17 18:08 bureau.csv
-rw-r--r-- 1 karti 197609 375592889 May 17 18:08 bureau_balance.csv
-rw-r--r-- 1 karti 197609 424582605 May 17 18:10 credit_card_balance.csv
-rw-r--r-- 1 karti 197609     37383 May 20 19:55 HomeCredit_columns_description.csv
-rw-r--r-- 1 karti 197609 723118349 May 17 18:13 installments_payments.csv
-rw-r--r-- 1 karti 197609 392703158 May 17 18:14 POS_CASH_balance.csv
-rw-r--r-- 1 karti 197609 404973293 May 17 18:15 previous_application.csv
-rw-r--r-- 1 karti 197609    536202 May 17 18:06 sample_submission.csv


- application_{train|test}.csv

This is the main table, broken into two files for Train (**with TARGET**) and Test (without TARGET).
Static data for all applications. **One row represents one loan in our data sample.**

- bureau.csv

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

- bureau_balance.csv

Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

- POS_CASH_balance.csv

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

- credit_card_balance.csv

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

- previous_application.csv

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.

- installments_payments.csv

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

- HomeCredit_columns_description.csv

This file contains descriptions for the columns in the various data files.

![](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

# Data Pre-processing

In [3]:
df_application_train = pd.read_csv("../data/application_train.csv")

In [4]:
df_application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_application_test = pd.read_csv("../data/application_test.csv")

In [6]:
df_application_test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


** JOIN 1 **

In [7]:
df_bureau = pd.read_csv("../data/bureau.csv")
df_bureau_balance = pd.read_csv("../data/bureau_balance.csv")

In [8]:
df_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [9]:
df_bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [10]:
df_join_bureau_bureaubalance = df_bureau.merge(df_bureau_balance, on="SK_ID_BUREAU")

In [11]:
print("Dimensions: df_bureau", df_bureau.shape)
print("Dimensions: df_bureau_balance", df_bureau_balance.shape)
print("Dimensions: df_join_bureau_bureaubalance", df_join_bureau_bureaubalance.shape)

Dimensions: df_bureau (1716428, 17)
Dimensions: df_bureau_balance (27299925, 3)
Dimensions: df_join_bureau_bureaubalance (24179741, 19)


** JOIN 2: df_join_bureau_bureaubalance with df_application_train **

In [12]:
df_applicationTrain_join_bureauBureaubalance = df_application_train.merge(df_join_bureau_bureaubalance, on="SK_ID_CURR")

In [14]:
print("Dimensions: df_join_bureau_bureaubalance", df_join_bureau_bureaubalance.shape)
print("Dimensions: df_application_train", df_application_train.shape)
print("Dimensions: df_applicationTrain_join_bureauBureaubalance", df_applicationTrain_join_bureauBureaubalance.shape)

Dimensions: df_join_bureau_bureaubalance (24179741, 19)
Dimensions: df_application_train (307511, 122)
Dimensions: df_applicationTrain_join_bureauBureaubalance (14701612, 140)


In [15]:
# df_applicationTrain_join_bureauBureaubalance.to_csv("../transformed_data/applicationTrain_join_bureauBureaubalance.csv")

** JOIN 3: df_join_bureau_bureaubalance with df_application_test **

In [16]:
df_applicationTest_join_bureauBureaubalance = df_application_test.merge(df_join_bureau_bureaubalance, on="SK_ID_CURR")

In [17]:
print("Dimensions: df_join_bureau_bureaubalance", df_join_bureau_bureaubalance.shape)
print("Dimensions: df_application_test", df_application_test.shape)
print("Dimensions: df_applicationTrain_join_bureauBureaubalance", df_applicationTrain_join_bureauBureaubalance.shape)

Dimensions: df_join_bureau_bureaubalance (24179741, 19)
Dimensions: df_application_test (48744, 121)
Dimensions: df_applicationTrain_join_bureauBureaubalance (14701612, 140)


# Model Building

## Encode categorical columns

In [7]:
arr_categorical_columns = df_application_train.select_dtypes(['object']).columns
for var_col in arr_categorical_columns:
    df_application_train[var_col] = df_application_train[var_col].astype('category').cat.codes
gc.collect()
arr_categorical_columns = df_application_test.select_dtypes(['object']).columns
for var_col in arr_categorical_columns:
    df_application_test[var_col] = df_application_test[var_col].astype('category').cat.codes

## Train-Validation Split

In [8]:
input_columns = df_application_train.columns
input_columns = input_columns[input_columns != 'TARGET']
target_column = 'TARGET'

X = df_application_train[input_columns]
y = df_application_train[target_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [25]:
def score(params):
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    
    dtrain = xgb.DMatrix(X_train, y_train)
    dtest = xgb.DMatrix(X_test, y_test)
    watchlist = [(dtrain, 'train'), (dtest, 'valid')]
    
    xgb_model = xgb.train(xgb_params, dtrain, 270, evals=watchlist, verbose_eval=100)
    predictions = xgb_model.predict(dtest, ntree_limit=xgb_model.best_iteration)
    
    score = roc_auc_score(y_test, predictions)
    print("\tScore {0}\n\n".format(score))
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}

In [26]:
def optimize(evals, cores=-1, optimizer=tpe.suggest, random_state=0):
    
    space = {
        'n_estimators': hp.quniform('n_estimators', 50, 1000, 1),
        'eta': hp.quniform('eta', 0.025, 0.25, 0.025), 
        'max_depth':  hp.choice('max_depth', np.arange(1, 20, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'alpha' :  hp.quniform('alpha', 0, 20, 1),
        'lambda': hp.quniform('lambda', 1, 2, 0.1),
        'nthread': -1,
        'objective': 'binary:logistic',
        'booster': 'gbtree',
        'eval_metric': 'auc',
        'seed': random_state
    }
    
    best = fmin(score, space, algo=tpe.suggest, max_evals=evals)
    return best

In [27]:
best_param = optimize(evals = 5, optimizer=tpe.suggest)
print("------------------------------------")
print("The best hyperparameters are: ", "\n")
print(best_param)


[0]	train-auc:0.727403	valid-auc:0.713529
[100]	train-auc:0.873175	valid-auc:0.754423
[200]	train-auc:0.922062	valid-auc:0.753007
[269]	train-auc:0.944288	valid-auc:0.750696
	Score 0.750762807449496


[0]	train-auc:0.727403	valid-auc:0.713529
[100]	train-auc:0.873175	valid-auc:0.754423
[200]	train-auc:0.922062	valid-auc:0.753007
[269]	train-auc:0.944288	valid-auc:0.750696
	Score 0.750762807449496


[0]	train-auc:0.727403	valid-auc:0.713529
[100]	train-auc:0.873175	valid-auc:0.754423
[200]	train-auc:0.922062	valid-auc:0.753007
[269]	train-auc:0.944288	valid-auc:0.750696
	Score 0.750762807449496


[0]	train-auc:0.727403	valid-auc:0.713529
[100]	train-auc:0.873175	valid-auc:0.754423
[200]	train-auc:0.922062	valid-auc:0.753007
[269]	train-auc:0.944288	valid-auc:0.750696
	Score 0.750762807449496


[0]	train-auc:0.727403	valid-auc:0.713529
[100]	train-auc:0.873175	valid-auc:0.754423
[200]	train-auc:0.922062	valid-auc:0.753007
[269]	train-auc:0.944288	valid-auc:0.750696
	Score 0.7507628074494

In [15]:
xgb_params = {
    'seed': 42,
    'booster': 'gbtree',   #Tree-based models
    'silent': 0,           #Messages would be printed
    'n_thread': -1,        #-1: all cores are used
    'objective': 'binary:logistic',
    'eval_metric': 'auc', 

    'eta': 0.15000000000000002,            #Learning rate- makes the model more robust via shrinkage
    'min_child_weight': 12, #Tradeoff b/n over and underfitting
    'max_depth': 5,        #Tradeoff b/n over and underfitting
    'subsample': 0.8500000000000001,      #Fraction of observations to be randomly sampled for each tree
    'alpha': 20.0, 
    'colsample_bytree': 0.6000000000000001,
    'gamma': 0.75, 
    'lambda': 1.6,
    'n_estimators': 281.0
    
}

In [17]:
watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_test, y_test), 'valid')]
model = xgb.train(xgb_params, xgb.DMatrix(X_train, y_train), 270, watchlist, maximize=True, verbose_eval=100)

[0]	train-auc:0.727403	valid-auc:0.713529
[100]	train-auc:0.873175	valid-auc:0.754423
[200]	train-auc:0.922062	valid-auc:0.753007
[269]	train-auc:0.944288	valid-auc:0.750696


In [16]:
# #Final Model
watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_test, y_test), 'valid')]
model = xgb.train(xgb_params, xgb.DMatrix(X, y), 270, watchlist, maximize=True, verbose_eval=100)

[0]	train-auc:0.629866	valid-auc:0.633053
[100]	train-auc:0.776694	valid-auc:0.777078
[200]	train-auc:0.793163	valid-auc:0.793941
[269]	train-auc:0.802092	valid-auc:0.80334


In [27]:
df_predict = model.predict(xgb.DMatrix(df_application_test), ntree_limit=model.best_ntree_limit)

In [28]:
submission = pd.DataFrame()
submission["SK_ID_CURR"] =  df_application_test["SK_ID_CURR"]
submission["TARGET"] =  df_predict

submission.to_csv("../submissions/model_1_xgbstarter_updatedParams_v1.csv", index=False)

In [29]:
submission.shape

(48744, 2)

In [25]:
def score(params):

    # #Reference: http://sbern.com/2017/05/03/optimizing-hyperparameters-with-hyperopt/
    scores = []
    
    kf = StratifiedKFold(n_folds=20, random_state=RANDOM_STATE, shuffle = True)
    for i, (train_index, test_index) in enumerate(kf.split(X, y)):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]    
        
        xgb_train = xgb.DMatrix(X_train, label=y_train)
        xgb_val = xgb.DMatrix(X_val, label=y_val)
        watchlist = [(xgb_train, 'train'), (xgb_val, 'eval')]
        xgb_model = xgb.train(xgb_params, xgb_train, 1000, watchlist, verbose_eval=200) 
        
        lg = xgb_model.predict(xgb_val, ntree_limit=model.best_ntree_limit) 
        res = [np.argmax(lg[i]) for i in range(lg.shape[0])]
        scores.append(accuracy_score(y_val, res))
        print('XGBoost', scores[-1])
    
    score = np.mean(scores)
    
    print("############### Score: {0}".format(score))
    print("############### Prms: ", params)
    print('..........................')
    
    score = roc_auc_score(y_test, predictions)
    print("\tScore {0}\n\n".format(score))
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}
